In [1]:
import os


import gdal

import pygeoprocessing
from pygeoprocessing import routing

In [2]:
import netCDF4
import numpy as np

from datetime import datetime, timedelta
import datetime # Python standard library datetime module

from netCDF4 import num2date, date2num
from netCDF4 import Dataset,netcdftime # http://unidata.github.io/netcdf4-python/

#open netCDF file:
fin_name = 'data_2013_test/temp_sphumid_relhumid_2013_600mb.nc'
fin = netCDF4.Dataset(fin_name)
#print(f)

# list variable names
print(fin.variables.keys())

#
# Latitude and longitude
#

varname = 'longitude'
longitude = fin.variables[varname][:] # get values

varname = 'latitude'
latitude = fin.variables[varname][:] # get values

#
# Time
#

tname = 'time'

time_array = fin.variables[tname][:] # get values
t_unit = fin.variables[tname].units # get unit  'days since 1950-01-01T00:00:00Z'
t_cal = fin.variables[tname].calendar # get calendars 'gregorian'

date_array = num2date(time_array,units = t_unit,calendar = t_cal)

# make arrays to hold the dates
year  = np.zeros(len(date_array), dtype=np.int)
month = np.zeros(len(date_array), dtype=np.int)
day   = np.zeros(len(date_array), dtype=np.int)
hour  = np.zeros(len(date_array), dtype=np.int)

for i in range (0, len(date_array)):
    strtime  = str(date_array[i])
    year[i]  = int(strtime[0:4])
    month[i] = int(strtime[5:7])
    day[i]  = int(strtime[8:10])
    hour[i]  = int(strtime[11:13])

#
# Cliamte
#

#u't', u'q', u'r']

# temperature 't'
var_num = 3
varname = fin.variables.items()[var_num][0] 

t_temperature = fin.variables[varname][:] # get values which are already unpacked 


# specific humidity 'q'
var_num = 4
varname = fin.variables.items()[var_num][0] 

q_specific_hum = fin.variables[varname][:] # get values which are already unpacked 


# relative humidity 'q'
var_num = 5
varname = fin.variables.items()[var_num][0] 

r_relative_hum = fin.variables[varname][:] # get values which are already unpacked 

[u'longitude', u'latitude', u'time', u't', u'q', u'r']


In [3]:
print(longitude[0])
print(latitude[0])

71.75
38.75


In [4]:
# from: https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html


import gdal, ogr, os, osr
import numpy as np


def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Byte)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()


def main(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):
    reversed_arr = array[::-1] # reverse array so the tif looks like the array
    array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,reversed_arr) # convert array to raster


if __name__ == "__main__":
    rasterOrigin = (longitude[0],latitude[0])
    pixelWidth = longitude[1] - longitude[0]
    pixelHeight = latitude[1] - latitude[0]
    newRasterfn = 'test.tif'
    array = r_relative_hum[0,:,:]


    main(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array)

0